In [57]:
import pandas as pd
import numpy as np

In [58]:
dataPath = 'HEINENS.xlsx'
df = pd.ExcelFile(dataPath)
df.sheet_names

['EASTER 2026', 'VDAY 26', 'fall 25']

In [59]:
sheetName = 'EASTER 2026'
df = pd.read_excel(dataPath, sheet_name=sheetName)

# Basic Data Analysis of moving the Column Names
df = df.drop(columns=df.columns[0])

header_row_idx = df[df.iloc[:, 0] == "PRODUCT"].index[0]
df.columns = df.iloc[header_row_idx]
df = df.drop(range(header_row_idx + 1)).reset_index(drop=True)
df

1,PRODUCT,FARM,SIZE,BOX/\n TYPE,PRECIO/\nFINCA,TALLOS X BUNCH,CAPUCHON,PRECIO/\n BQT,FLETE\n (MIAMI),WET\n PACK,...,PRICE,PRICE\n/BUNCH,TRANSP\n/PALL,WP/Bqt,Cube Wet Pack,Price Per Cube,Price Per Piece,Fuel,Price Per Box,F.EEUU/ Bqt
0,BQT MESSY EASTER 20 ST,Much,QB2,QB2,6,1,0,6,3.07675,0,...,0,0,0,0,1.373034,2.18,0.5,0.448982,4.2962,0.42962
1,BQT EASTER GRATITUDE\n50cm- 13 Stems,MUCH,50,QB,4.93,1,0,4.93,2.085454,0,...,0,0,0,0,1.373034,2.18,0.5,0.448982,4.2962,0.42962
2,BQT EASTER PREMIUM\n50cm- 16 Stems,CULTIVOS,50,QB,7.64,1,0,7.64,3.237742,0,...,0,0,0,0,1.349727,2.18,0.5,0.441361,4.212983,0.421298
3,BQT FEELING EASTER MIX\n50cm 20 stems,MUCH,50,QB2,6.4,1,0,6.4,3.16395,0,...,0,0,0,0,1.373034,2.18,0.5,0.448982,4.2962,0.306871
4,RCB NATURAL EASTER\n50cm 3 stems,ROSADEX,50,QB,1.29,1,0,1.29,0.92522,0,...,0,0,0,0,2.888889,2.18,0.5,0.944667,10.471259,1.047126
5,TND MARSHMELLOW\n50cm 6 stems,DYED,50,QB,0.85,1,0,0.85,1.19155,0,...,0,0,0,0,2.888889,2.18,0.5,0.944667,10.471259,1.047126
6,BQT EASTER RAIBOW\n50cm- 12 Stems,ROSADEX,50,H1,3.99,1,0,3.99,3.023987,0,...,0,0,0,0,2.627835,2.18,0.5,0.859302,9.300702,0.93007
7,BULK EASTER NATURAL\n60cm 25 stems,ROSADEX,60,QB,0.38,1,0,0.38,4.10784,0,...,0,0,0,0,1.668618,2.18,0.5,0.545638,5.382358,0.538236
8,SLAY: Aster+ Fever Few,MUCH,60,QB2,3.45,1,0,3.45,1.931267,1.179167,...,5.75,0.479167,0.7,1.179167,2.888889,2.18,0.5,0.944667,10.471259,0.872605
9,SLAY: SUnFlower+Lepidium,MUCH,60,QB2,3.35,1,0,3.35,2.1453,1.275,...,5.75,0.575,0.7,1.275,2.888889,2.18,0.5,0.944667,10.471259,1.047126


In [60]:
def normalize_cols(cols: pd.Index) -> pd.Index:
    """Collapse whitespace/newlines, trim, upper-case, replace spaces with underscores."""
    return (
        cols.astype(str)
            .str.replace(r"\s+", " ", regex=True)
            .str.strip()
            .str.upper()
            .str.replace(" ", "_")
    )
df.columns = normalize_cols(df.columns)  # e.g., "COSTO\n TOTAL" -> "COSTO_TOTAL", "PRICE \nCLIENTE" -> "PRICE_CLIENTE"

df.columns

Index(['PRODUCT', 'FARM', 'SIZE', 'BOX/_TYPE', 'PRECIO/_FINCA',
       'TALLOS_X_BUNCH', 'CAPUCHON', 'PRECIO/_BQT', 'FLETE_(MIAMI)',
       'WET_PACK', 'FREIGHT', 'COSTO_TOTAL', 'PRICE_CLIENTE', 'UNIDADES',
       'BUNCH_X_CAJA', 'TOTAL_CAJA', 'TALLOS_CAJA', 'LENGTH', 'WIDTH',
       'HEIGHT', 'VOLUME', 'ROUNDED_VOLUME', 'PRECIO_KILO', 'PRECIO_CAJA',
       'EXTRAS', 'DUTIES', 'FLETE_/BQT', 'PACK', 'WET_PACK_COMPANY', 'HT',
       'WD', 'DP', 'CUBE', 'PRICE', 'PRICE_/BUNCH', 'TRANSP_/PALL', 'WP/BQT',
       'CUBE_WET_PACK', 'PRICE_PER_CUBE', 'PRICE_PER_PIECE', 'FUEL',
       'PRICE_PER_BOX', 'F.EEUU/_BQT'],
      dtype='object', name=1)

In [61]:
df['FARM'] = df['FARM'].str.upper()

costoTotal = sum([df['PRECIO/_FINCA'],df['FLETE_(MIAMI)'],df['WET_PACK'],df['FREIGHT']])

margin = 0.15

formulaMargen = (1/ -1 * (-1 - 0.85)) - 1

df['PRICE_CLIENTE'] = costoTotal * (1 + formulaMargen)

df['PRICE_CLIENTE'] = pd.to_numeric(df['PRICE_CLIENTE'])
df['PRICE_CLIENTE'] = df['PRICE_CLIENTE'].round(2)
print(costoTotal[0])
print(df['PRICE_CLIENTE'][0])
pd.to_numeric(df['FUEL']).round(2)


9.50636997
17.59


0     0.45
1     0.45
2     0.44
3     0.45
4     0.94
5     0.94
6     0.86
7     0.55
8     0.94
9     0.94
10    0.41
11    0.41
12    0.45
13    0.45
14    0.45
15    0.45
16    0.45
17    0.45
18    0.45
19    0.45
Name: FUEL, dtype: float64

In [62]:
def round_up(value, decimals=2):
    factor = 10 ** decimals
    return np.ceil(value * factor) / factor

# Flete Miami

# Volume
length = df['LENGTH']
width = df['WIDTH']
height = df['HEIGHT']
ratioFlete = 6000

df['VOLUME'] = (length * width * height) / ratioFlete  # Volume
df['ROUNDED_VOLUME'] = np.ceil(df['VOLUME'])    # Rounded Volume

roundedVol = df['ROUNDED_VOLUME']

# Precio Caja
df['PRECIO_CAJA'] = roundedVol * df['PRECIO_KILO']

# Duties
dutyMultiplier = 0.218
precioBQT = df['PRECIO/_BQT']
bunchCaja = df['BUNCH_X_CAJA']
df['DUTIES'] = ( precioBQT * bunchCaja) * dutyMultiplier


#FLETE
precioKilo = df['PRECIO_KILO']
precioCaja = roundedVol * precioKilo
extrasBuffer = df['EXTRAS']
duties = df['DUTIES']
totalCaja = df['TOTAL_CAJA']
df['FLETE_/BQT'] = pd.to_numeric(((precioCaja + extrasBuffer + duties) / totalCaja)).round(2)
df['FUEL']

0     0.448982
1     0.448982
2     0.441361
3     0.448982
4     0.944667
5     0.944667
6     0.859302
7     0.545638
8     0.944667
9     0.944667
10    0.409835
11    0.409835
12    0.448982
13    0.448982
14    0.448982
15    0.448982
16    0.448982
17    0.448982
18    0.448982
19    0.448982
Name: FUEL, dtype: object

In [63]:
# WET PACKS

# HT
wetPackConst = 2.54
df['HT'] = df['LENGTH'] / wetPackConst

# WD
df['WD'] = df['WIDTH'] / wetPackConst

# DP
df['DP'] = df['HEIGHT'] / wetPackConst

# CUBE
cubeConst = 1728
ht = df['HT']
wd = df['WD']
dp = df['DP']
df['CUBE'] = pd.to_numeric(((ht * wd * dp) / cubeConst)).round(2)

# Price Bunch
wetPackPrice = df['PRICE']
wetPackSize = df['PACK']

df['PRICE_/BUNCH'] = pd.to_numeric(( wetPackPrice/ wetPackSize)).round(2)


# WP/BQT
priceBunch = df['PRICE_/BUNCH']
transportPallet = df['TRANSP_/PALL']
df['WP/BQT'] = priceBunch + transportPallet
df['CUBE']

0     1.37
1     1.37
2     1.35
3     1.37
4     1.67
5     1.67
6     2.63
7     1.67
8     1.37
9     1.37
10    1.25
11    1.25
12    1.37
13    1.37
14    1.37
15    1.37
16    1.37
17    1.37
18    1.37
19    1.37
Name: CUBE, dtype: float64

In [66]:
# Freight
cube = df['CUBE']
df['CUBE_WET_PACK'] = cube.round(2)

# Price Per Piece
cubeWetPack = pd.to_numeric(df['CUBE_WET_PACK'])
pricePerCube = 2.18
pricePerPiece = 0.50
fuelConst = 0.30
df['FUEL'] = ((pricePerCube * pricePerPiece * cubeWetPack) * fuelConst).round(2)



# Price Per Box
fuel = df['FUEL']

df['PRICE_PER_BOX'] = pd.to_numeric((cubeWetPack * (pricePerCube + pricePerPiece + fuel))).round(2)
df['PRICE_PER_BOX'] = df['PRICE_PER_BOX'].apply(lambda x: round_up(x, 2))
df['PRICE_PER_BOX']

# FEEUU / BQT

0     4.29
1     4.29
2     4.21
3     4.29
4     5.39
5     5.39
6     9.31
7     5.39
8     4.29
9     4.29
10    3.86
11    3.86
12    4.29
13    4.29
14    4.29
15    4.29
16    4.29
17    4.29
18    4.29
19    4.29
Name: PRICE_PER_BOX, dtype: float64

In [65]:
df.columns

Index(['PRODUCT', 'FARM', 'SIZE', 'BOX/_TYPE', 'PRECIO/_FINCA',
       'TALLOS_X_BUNCH', 'CAPUCHON', 'PRECIO/_BQT', 'FLETE_(MIAMI)',
       'WET_PACK', 'FREIGHT', 'COSTO_TOTAL', 'PRICE_CLIENTE', 'UNIDADES',
       'BUNCH_X_CAJA', 'TOTAL_CAJA', 'TALLOS_CAJA', 'LENGTH', 'WIDTH',
       'HEIGHT', 'VOLUME', 'ROUNDED_VOLUME', 'PRECIO_KILO', 'PRECIO_CAJA',
       'EXTRAS', 'DUTIES', 'FLETE_/BQT', 'PACK', 'WET_PACK_COMPANY', 'HT',
       'WD', 'DP', 'CUBE', 'PRICE', 'PRICE_/BUNCH', 'TRANSP_/PALL', 'WP/BQT',
       'CUBE_WET_PACK', 'PRICE_PER_CUBE', 'PRICE_PER_PIECE', 'FUEL',
       'PRICE_PER_BOX', 'F.EEUU/_BQT'],
      dtype='object', name=1)